# [Day 7](https://adventofcode.com/2022/day/7)

## File system classes

In [1]:
class File:
    def __new__(cls, name, parent = None, size = 0):
        file = super(File, cls).__new__(cls)
        file.name = name
        file.parent = parent
        file.size = size
        file.children = dict()
        return file   
    def fullname(self):
        fullname = []
        if self.children:
            fullname.append('')
        curr = self
        while curr:
            fullname.append(curr.name)
            curr = curr.parent
        return '/'.join(reversed(fullname))
        
class FileSystem:
    def __init__(self):
        self.root = File('/')
        self.current = self.root
    def mkdir(self, name):
        if name not in self.current.children:
            self.current.children[name] = File(name, self.current)
    def create(self, name, size):
        if name not in self.current.children:
            self.current.children[name] = File(name, self.current)
        self.current.children[name].size = size
    def cd(self, param):
        if param == '/':
            self.current = self.root
        elif param == '..':
            self.current = self.current.parent
        elif param in self.current.children:
            self.current = self.current.children[param]
        else:
            raise Exception("unknown subdirectory")
        
    def postorder(file, fn):
        for f in file.children.values():
            FileSystem.postorder(f, fn)
        fn(file)
        
    def __du(self, sizes, file):
        if file == self.root:
            return
        if file.children: # dir
            sizes[file.parent] = sizes.get(file.parent, 0) + sizes.get(file, 0)
        else: # file or empty dir
            sizes[file.parent] = sizes.get(file.parent, 0) + file.size
            
    def du(self):
        sizes = dict()
        FileSystem.postorder(self.root, lambda f : self.__du(sizes, f))
        return sizes

## Read input into fs

In [2]:
with open('input.txt', 'r') as file:
    lines = file.read().strip().split('\n');

fs = FileSystem()
curdir = None
for line in lines:
    line = line.split(' ')
    if line[0] == '$':
        if line[1] == 'cd':
            fs.cd(line[2])
    elif line[0] == 'dir':
        fs.mkdir(line[1])
    else:
        fs.create(line[1], int(line[0]))

## Part 1

In [3]:
du = fs.du()
sum([du[x] for x in du.keys() if du[x] <= 100000])

1477771

## Part 2

In [4]:

totalSpace = 70000000
usedSpace = du[fs.root]
unusedSpace = totalSpace - usedSpace
min([du[x] for x in du.keys() if du[x] >= 30000000 - unusedSpace])

3579501

# No tree solution

In [5]:
with open('input.txt', 'r') as file:
    lines = file.read().strip().split('\n');

du = dict()
curdir = ['/']
for line in lines:
    line = line.split(' ')
    if line[0] == '$':
        if line[1] == 'cd':
            if line[2] == '/':

                curdir = ['/']
            elif line[2] == '..':
                curdir = curdir[:-1]
            else:
                curdir.append(line[2])
    elif line[0] != 'dir':
        for i in range(len(curdir)+1):
            du['/'.join(curdir[:i])] = du.get('/'.join(curdir[:i]), 0) + int(line[0])

display(f"Part 1: {sum([du[x] for x in du.keys() if du[x] <= 100000])}")
        
totalSpace = 70000000
usedSpace = du['/']
unusedSpace = totalSpace - usedSpace

display(f"Part 2: {min([du[x] for x in du.keys() if du[x] >= 30000000 - unusedSpace])}")

'Part 1: 1477771'

'Part 2: 3579501'